In [389]:
import time
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

In [390]:
STEPS = 500
RANGE = 250
N_PERSONAS = 1000
LONG_CELDA_EN_M = 0.4
CELDAS_DE_DISTANCIA = 5

In [391]:
class Persona:
    q_persona = 0
    
    def __init__(self, x_init, y_init, tablero, contagiado, α, β, verbose=False):
        self.numero = Persona.q_persona
        Persona.q_persona += 1
        self.tablero = tablero
        self.contagiado = contagiado
        self.steps_enfermo = 0
        self.x = x_init
        self.y = y_init
        self.verbose = verbose
        self.α = α
        self.β = β

        
    def get_position(self):
        return self.x, self.y
    
    def esta_contagiado(self):
        return self.contagiado
        
    def step_normal(self):
        movimientos = self.tablero.get_movimientos_posibles(self.x, self.y)
        idx = np.argmax(np.random.uniform(0, LONG_CELDA_EN_M, len(movimientos)))
        self.x, self.y = movimientos[idx]
        if self.verbose:
            print("({}) Posicion x: {}".format(self.numero, self.x))
            print("({}) Posicion y: {}".format(self.numero, self.y))
        return self.x, self.y
    
    def propagar_contagios(self):
        if self.contagiado:
            self.steps_enfermo += 1
            if self.steps_enfermo >= self.α:
                if (np.random.rand() <= β):
                    self.contagiado = False
                    self.steps_enfermo = 0
        else:
            self.contagiado = self.tablero.propagar_contagios_por_cercania(self.x, self.y, CELDAS_DE_DISTANCIA)
#             if self.contagiado:
#                 self.steps_enfermo = 0
        
class Tablero:
    # α: tiempo de recuperacion
    # B: probabilidad de sanar después del tiempo de recuperacion
    def __init__(self, n_dim, n_personas, tasa_contagio_inicial, α, β, verbose=False):
        self.verbose = verbose
        self.dim = n_dim
        self.personas = self.__init_personas(n_personas, α, β)
        self.tasa_contagio = tasa_contagio_inicial
    
    def __init_personas(self, n_personas, α, β):
        def get_init_positions(n_persons):
            indexes = [(i, j) for i in range(0, self.dim) for j in range(0, self.dim)]
            random.shuffle(indexes)
            return indexes[:n_persons]
        
        def generate_persons(pos, α, β):
            persons = []
            n = np.random.rand(len(pos))
            s = np.random.rand(len(pos))
            for (x_init, y_init), ni, si in zip(pos, n, s):
                contagiado = (si <= 0.03)
                if ni <= 0.1:
                    persons.append(PersonaC(x_init, y_init, self, contagiado,
                                            α, β, self.verbose))
                elif ni > 0.1 and ni <= 0.3:
                    persons.append(PersonaB(x_init, y_init, self, contagiado,
                                            α, β, self.verbose))
                else:
                    persons.append(PersonaA(x_init, y_init, self, contagiado,
                                            α, β, self.verbose))
            return persons
        
        init_positions = get_init_positions(n_personas)
        return generate_persons(init_positions, α, β)
        
    def get_movimientos_posibles(self, x, y):
        movs = [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]
        movs = list(filter(lambda x: x[0] > 0 and x[0] <= self.dim and x[1] > 0 and x[1] <= self.dim, movs))
        actual_positions = [p.get_position() for p in self.personas]
        possible_movs = set(movs) - set(actual_positions)
        return list(possible_movs)
    
    def propagar_contagios_por_cercania(self, x, y, limite):
        def esta_suficientemente_cerca(x, y, actual_contagiados, limite):
            resultado = []
            for c in actual_contagiados:
                distancia_en_x = abs(c[0] - x)
                distancia_en_y = abs(c[1] - y)
                resultado.append((distancia_en_x + distancia_en_y) < 5)
            return resultado
        
        actual_contagiados = [p.get_position() for p in self.personas if p.esta_contagiado()]
        # Acá podríamos hacerlo para todos, pero lo voy a hacer una única vez: si tengo 1 contagiado o 10 cerca,
        # va a ser lo mismo. Puede ser algo a mejorar #
        if any(esta_suficientemente_cerca(x, y, actual_contagiados, limite)):
            return( np.random.rand() <= self.tasa_contagio )
            
        return False

    def siguiente_ronda(self):
        for p in self.personas:
            p.step()
            p.propagar_contagios()
        return self.personas

In [392]:
class PersonaC(Persona):
    def __init__(self, x_inicial, y_inicial, tablero, contagiado, α, β, verbose=False):
        super().__init__(x_inicial, y_inicial, tablero, contagiado, α, β, verbose)
        self.tipo = 'C'
#         self.hex_color = '#49111C' # bordó
        self.hex_color = '#3d405b' # violeta
        self.contador = 0
        
    def step(self):
        return self.step_normal()
    
    def step_normal(self):
        self.contador += 1
        if self.contador == 4:
            self.contador = 0
            movimientos = self.tablero.get_movimientos_posibles(self.x, self.y)
            if len(movimientos) > 0:
                idx = np.argmax(np.random.uniform(0, LONG_CELDA_EN_M, len(movimientos)))
                self.x, self.y = movimientos[idx]
                if self.verbose:
                    print("({}) Posicion x: {}".format(self.numero, self.x))
                    print("({}) Posicion y: {}".format(self.numero, self.y))
        return self.x, self.y

    
class PersonaB(Persona):
    def __init__(self, x_inicial, y_inicial, tablero, contagiado, α, β, verbose=False):
        super().__init__(x_inicial, y_inicial, tablero, contagiado, α, β, verbose)
        self.tipo = 'B'
#         self.hex_color = '#FF5A5F' # naranja1
        self.hex_color = '#e07a5f' # naranja2
        self.contador = 0

    def step(self):
        return self.step_normal()
    
    def step_normal(self):
        self.contador += 1
        if self.contador == 2:
            self.contador = 0
            movimientos = self.tablero.get_movimientos_posibles(self.x, self.y)
            idx = np.argmax(np.random.uniform(0, LONG_CELDA_EN_M, len(movimientos)))
            self.x, self.y = movimientos[idx]
            if self.verbose:
                print("({}) Posicion x: {}".format(self.numero, self.x))
                print("({}) Posicion y: {}".format(self.numero, self.y))
        return self.x, self.y
    
class PersonaA(Persona):
    def __init__(self, x_inicial, y_inicial, tablero, contagiado, α, β, verbose=False):
        super().__init__(x_inicial, y_inicial, tablero, contagiado, α, β, verbose)
        self.tipo = 'A'
#         self.hex_color = '#087E8B' # celeste
        self.hex_color = '#81b29a' # verde

    def step(self):
        return self.step_normal()

In [393]:
colores_por_enfermedad = {True: '#ef233c', False: '#8ecae6'}

def animate_persons(n_step, tablero, lines, por_enfermedad=False):
    new_lines = []
    persons = tablero.siguiente_ronda()
    for person, line in zip(persons, lines):
        x, y = person.get_position()
        line.set_data([x*LONG_CELDA_EN_M], [y*LONG_CELDA_EN_M])
        if not por_enfermedad:
            line.set_color(person.hex_color)
        else:
            line.set_color(colores_por_enfermedad[person.esta_contagiado()])
        new_lines.append(line)
    return new_lines

In [394]:
def gen_lines(ax, n_personas):
#     cmap = plt.cm.get_cmap('hsv', n_personas)
    lines = [ax.plot([], [], marker='o', markersize=9)[0] for n in range(n_personas)]
    return lines

In [395]:
inicio = time.time()

#Set up ax
fig = plt.figure(figsize=(21, 10))
ax = plt.axes(xlim=(0, RANGE*LONG_CELDA_EN_M), ylim=(0, RANGE*LONG_CELDA_EN_M)) 
lines = gen_lines(ax, N_PERSONAS)
ax.set_xlabel('Steps', fontsize=18)
ax.set_ylabel('Value', fontsize=18)
ax.tick_params(labelsize=16)
ax.grid(True, which='major', linestyle='--', color='black', alpha=0.4)

#Set up tablero
tasa_de_contagio = 0.65
α = STEPS/20
β = 0.2
ax.set_title('Simulación del contagio de una enfermedad', fontsize=22)
tablero = Tablero(RANGE, N_PERSONAS, tasa_de_contagio, α, β)

# call the animator	 
animar_por_enfermedad = True
anim = animation.FuncAnimation(fig, animate_persons, fargs = (tablero, lines, animar_por_enfermedad),
                               frames=STEPS, interval=295, blit=True)

anim.save('videos/randomwalk_alpha={0}_beta={1}.mp4'.format(α, β))
plt.close()
t = time.time() - inicio

print("Random walk guardado. Tardó {0} segundos, {1} minutos".format(t, t/60))

Random walk guardado. Tardó 482.92184019088745 segundos, 8.04869733651479 minutos
